###Clone the `diffusers` repository

In [ ]:
!git clone -b controlnet-inpaint-training https://github.com/NegarMov/diffusers.git

Cloning into 'diffusers'...
remote: Enumerating objects: 30146, done.
remote: Counting objects: 100% (6328/6328), done.
remote: Compressing objects: 100% (537/537), done.
remote: Total 30146 (delta 6076), reused 5792 (delta 5791), pack-reused 23818
Receiving objects: 100% (30146/30146), 22.30 MiB | 21.40 MiB/s, done.
Resolving deltas: 100% (22357/22357), done.


In [ ]:
%cd diffusers

/content/diffusers


In [ ]:
!pip install -e .

Obtaining file:///content/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.9 MB/s eta 0:00:00
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.22.0.dev0-0.editable-py3-none-any.whl size=10643 sha256=72b894fa7bb6b5df6e35392cc25964dc30217168202b36ce6d850105b745b634
  Stored in directory: /tmp/pip-ephem-wheel-cache-b6wpwq88/wheels/95/c5/3b/e1b4269f8a2584de57e75f949a185b48fc4144e9a91fc9965a
Successfully built diffusers


###Install the dependencies

In [ ]:
%cd examples/controlnet

/content/diffusers/examples/controlnet


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00


In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

###Train the model

In [ ]:
%env INPAINT_MODEL_DIR=runwayml/stable-diffusion-inpainting
%env MODEL_DIR=SG161222/Realistic_Vision_V3.0_VAE
%env OUTPUT_DIR=models/control_sd15

env: INPAINT_MODEL_DIR=runwayml/stable-diffusion-inpainting
env: MODEL_DIR=SG161222/Realistic_Vision_V3.0_VAE
env: OUTPUT_DIR=models/control_sd15


In [ ]:
!rm -rf models
!mkdir models

In [ ]:
!accelerate launch train_controlnet_inpaint.py \
 --pretrained_inpaint_model_name_or_path=$INPAINT_MODEL_DIR \
 --pretrained_model_name_or_path=$MODEL_DIR \
 --output_dir=$OUTPUT_DIR \
 --dataset_name=NegarMov/DF_segmented_mask \
 --image_column=flawless \
 --conditioning_image_column=reference \
 --mask_image_column=mask \
 --distorted_image_column=flawless \
 --caption_column=prompt \
 --resolution=512 \
 --learning_rate=1e-6 \
 --train_batch_size=1 \
 --gradient_accumulation_steps=4 \
 --gradient_checkpointing \
 --use_8bit_adam \
 --enable_xformers_memory_efficient_attention \
 --max_train_steps=7000 \
 --resume_from_checkpoint=latest \
 --checkpointing_steps=500

2023-09-17 11:37:57.977603: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
09/17/2023 11:37:59 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'rescale_betas_zero_snr', 'clip_sample_range', 'dynamic_thresholding_ratio', 'thresholding', 'sample_max_value', 'timestep_spacing', 'prediction_type'} was not found in config. Values will be initialized to default values.
{'force_upcast', 'scaling_factor'} was not found in config. Values will be initialized to default values.
{'timestep_post_act', 'transformer_layers_per_block', 'num_attention_heads', 'encoder_hid_dim', 'time_embedding_act_fn', 'class_embeddings_concat', 'only_cross_attention', 'conv_in_kernel', 'addition_time_embed_dim', 'encoder

In [ ]:
# !accelerate launch train_controlnet.py \
#  --pretrained_model_name_or_path=$MODEL_DIR \
#  --output_dir=$OUTPUT_DIR \
#  --dataset_name=NegarMov/DHI_test \
#  --image_column=flawless \
#  --conditioning_image_column=reference \
#  --caption_column=prompt \
#  --resolution=512 \
#  --learning_rate=1e-6 \
#  --train_batch_size=1 \
#  --gradient_accumulation_steps=4 \
#  --gradient_checkpointing \
#  --use_8bit_adam \
#  --enable_xformers_memory_efficient_attention \
#  --num_train_epochs=50 \
#  --checkpointing_steps=1000

###Performing inference with the trained model

In [ ]:
import sys
sys.path.append('/content/diffusers/src')

from diffusers import StableDiffusionControlNetInpaintPipeline, ControlNetModel, UniPCMultistepScheduler, DDIMScheduler
from diffusers.utils import load_image
import torch
import os

CHECK_POINT = 500

base_model_path = os.environ.get("INPAINT_MODEL_DIR")
# TODO: load this path using env
controlnet_path = f"/content/diffusers/examples/controlnet/models/control_sd15/checkpoint-{CHECK_POINT}/controlnet"

controlnet = ControlNetModel.from_pretrained(controlnet_path, torch_dtype=torch.float16)

pipe = StableDiffusionControlNetInpaintPipeline.from_pretrained(
    base_model_path, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
)

# speed up diffusion process with faster scheduler and memory optimization
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
# remove following line if xformers is not installed or when using Torch 2.0.
pipe.enable_xformers_memory_efficient_attention()
# memory optimization.
pipe.enable_model_cpu_offload()

vae/diffusion_pytorch_model.safetensors not found


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_inpaint.StableDiffusionControlNetInpaintPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
# load the images
distorted_image = load_image("/content/distorted.jpg")
flawless_image = load_image("/content/flawless.jpg")
control_image = load_image("/content/reference.jpg")
mask_image = load_image("/content/mask.jpg")
prompt = "high quality, detailed, professional photograph of a woman standing"

In [ ]:
# generate image
generator = torch.manual_seed(8)
output = pipe(
    prompt,
    num_inference_steps=30,
    generator=generator,
    image=flawless_image,
    mask_image=mask_image,
    control_image=control_image,
).images[0]
output.save(f"/content/output_flawless_{CHECK_POINT}.jpg")

output = pipe(
    prompt,
    num_inference_steps=30,
    generator=generator,
    image=distorted_image,
    mask_image=mask_image,
    control_image=control_image,
).images[0]
output.save(f"/content/output_distorted_{CHECK_POINT}.jpg")

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]